# CS483 - Colab 5
## PageRank

In [1]:
from IPython.display import Image
print("Colab 5 Mascot")
Image(url='https://media.giphy.com/media/cCOVfFwDI3awdse5A3/giphy.gif',width=150)

Colab 5 Mascot


### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks ([tutorial](https://networkx.org/documentation/stable/tutorial.html)).

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [5]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

#### Function to print graph information

In [6]:
def print_graph_info(G, directed=True):
  print("Number of nodes:", len(G.nodes))
  print("Number of edges:", len(G.edges))
  if directed:
    print("Average in-degree:", sum(dict(G.in_degree).values()) / len(G.nodes))
    print("Average out-degree:", sum(dict(G.out_degree).values()) / len(G.nodes))
  else:
    print("Average degree:", sum(dict(G.degree).values()) / len(G.nodes))

In [7]:
print_graph_info(G, True)

Number of nodes: 281903
Number of edges: 2312497
Average in-degree: 8.203165627893283
Average out-degree: 8.203165627893283


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

Print its information.

In [8]:
''' 3 lines of code in total expected. '''

# YOUR CODE HERE
weakly_connected_components = max(nx.weakly_connected_components(G), key=len)
max_weakly_connected_components = nx.subgraph(G, weakly_connected_components)
print_graph_info(max_weakly_connected_components, True)

Number of nodes: 255265
Number of edges: 2234572
Average in-degree: 8.753930229369479
Average out-degree: 8.753930229369479


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [9]:
''' 1 line of code in total expected. '''

# YOUR CODE HERE
rank = sorted(nx.pagerank(max_weakly_connected_components, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None).items(), key=lambda x:x[1])
print(rank[-1])

('89073', 0.011051585882434985)


In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using the parameters ```seed = 1``` and ```directed=False``` where applicable, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs. Print generated graph's information, if needed.


In [10]:
''' 6-8 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
n = 255265
p = 0.00008
rand_G = nx.fast_gnp_random_graph(n, p, seed=1, directed=False)
rank_rand_G = sorted(nx.pagerank(rand_G, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None).items(), key=lambda x:x[1])
print(rank_rand_G[-1])
print_graph_info(rand_G, False)

(150787, 7.458986619588909e-06)
Number of nodes: 255265
Number of edges: 2606386
Average degree: 20.421021291598926


In [11]:
m = 9
barabasi_albert_G = nx.barabasi_albert_graph(n, m, seed=1, initial_graph=None)
rank_barabasi_albert_G = sorted(nx.pagerank(barabasi_albert_G, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None).items(), key=lambda x:x[1])
print(rank_barabasi_albert_G[-1])
print_graph_info(barabasi_albert_G, False)

(11, 0.00041689766748208805)
Number of nodes: 255265
Number of edges: 2297304
Average degree: 17.99936536540458


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure.

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [18]:
''' 8-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
rank[-10:]

[('186750', 0.00314345200380852),
 ('234704', 0.0031708863624340614),
 ('225872', 0.0031736850016296342),
 ('105607', 0.0032305919516859047),
 ('67756', 0.003543473943866138),
 ('69358', 0.003753708143672675),
 ('134832', 0.00650923773721211),
 ('241454', 0.009829260884719991),
 ('226411', 0.010694113250567905),
 ('89073', 0.011051585882434985)]

In [19]:
rank_rand_G[-10:]

[(127392, 7.197456973720354e-06),
 (239691, 7.228833381965614e-06),
 (70124, 7.268730168861052e-06),
 (129032, 7.299830669613334e-06),
 (184745, 7.313541287755495e-06),
 (144307, 7.323021888645532e-06),
 (141505, 7.348699650919032e-06),
 (193475, 7.368176905091742e-06),
 (14579, 7.3974735409810506e-06),
 (150787, 7.458986619588909e-06)]

In [20]:
rank_barabasi_albert_G[-10:]

[(2, 0.0002820305369954235),
 (13, 0.0002832139282928614),
 (33, 0.00028542554346320257),
 (31, 0.0003024957900994401),
 (12, 0.0003051642675066239),
 (0, 0.00031311819116956204),
 (17, 0.000323435926854125),
 (10, 0.0003280249659834195),
 (6, 0.00034052486511954385),
 (11, 0.00041689766748208805)]

In [21]:
G_vec = np.array([x[1] for x in rank])
rand_vec = np.array([x[1] for x in rank_rand_G])
barabasi_albert_vec = np.array([x[1] for x in rank_barabasi_albert_G])

In [22]:
cosine_rand = np.dot(G_vec, rand_vec)/(np.linalg.norm(G_vec)*np.linalg.norm(rand_vec))
print("Cosine Similarity  between the web-Stanford graph and the random generated graph:", cosine_rand)
cosine_BA = np.dot(G_vec,barabasi_albert_vec)/(np.linalg.norm(G_vec)*np.linalg.norm(barabasi_albert_vec))
print("Cosine Similarity between the web-Stanford graph and the Barabási-Albert graph:", cosine_BA)

Cosine Similarity  between the web-Stanford graph and the random generated graph: 0.10395564703964394
Cosine Similarity between the web-Stanford graph and the Barabási-Albert graph: 0.6488673660967239


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!